# Create Azure Virtual Machine
Recall that in [the previous section](create_azure_services.ipynb), we created an IoT Edge device identity in Azure IoT Hub. In this section, we will be creating and configuring a virtual machine (VM) to act as our IoT Edge device.

## Get Global Variables

In [ ]:
from env_variables import *
resourceTags = "Owner={} Project=lva".format(userName)

## Check VM Quota

Verify that your subscription has enough quota to create your desired VM.

In [ ]:
output = !az vm list-sizes --subscription $azureSubscriptionId --location $resourceLocation --output json
sizes = ''.join(output)

import json
jsonDetails = json.loads(sizes)

for n in jsonDetails:
    if vm_type == n['name']:
        neededSpace = n['numberOfCores']
        print('\n' + n['name'])
        print("Number of Cores Required to Deploy: %s vCPUs" % neededSpace)

In [ ]:
!az account set --subscription $azureSubscriptionId
output = !az vm list-usage --location $resourceLocation --output json
output = ''.join(output)

In [ ]:
import json
jsonDetails = json.loads(output)

needed_size = vm_type.split('_')
needed_size = needed_size[1]
needed_size = ''.join([i for i in needed_size if not i.isdigit()])

quota_remaining = 0
for n in jsonDetails:
    if (needed_size in n['localName']):
        quota_remaining = int(n['limit']) - int(n['currentValue'])
        print('\n' + n['localName'])
        print("Quota Remaining: %s vCPUs" % quota_remaining)

Check if the quota remaining is greater than the size of the VM you wish to create.

If you do not enough quota, you can try any of the following:

* [Delete VMs](https://docs.microsoft.com/en-us/powershell/module/azurerm.compute/remove-azurermvm?view=azurermps-6.13.0) that might not be in use to free up quota.
* [Increase quota](https://docs.microsoft.com/en-us/azure/azure-portal/supportability/per-vm-quota-requests) limits.

## Create the Virtual Machine
The code snippets below will execute a shell script to create an Ubuntu VM in your Azure subscription. 

Before starting, we will be setting the parameters to be used for creating the VM. For more details on the process of creating a VM using Azure, see the tutorial on creating a [Linux VM on Azure CLI](https://docs.microsoft.com/en-us/azure/virtual-machines/linux/quick-create-cli) or the tutorial on creating a [Windows VM on Azure CLI](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/quick-create-cli).

In [ ]:
dns_name = iotDeviceId
public_ip_name = iotDeviceId + 'publicip'
vnet_name=iotDeviceId + 'vnet'
subnet_name=iotDeviceId + 'subnet'
vnet_prefix="192.168.0.0/16"
subnet_name="FrontEnd"
subnet_prefix="192.168.1.0/24"
nsg_name=iotDeviceId + 'nsg'
nic_name=iotDeviceId + 'nic'

# Static DNS name of the VM
vm_dns_name= iotDeviceId + "." + resourceLocation + ".cloudapp.azure.com"

In [ ]:
# Create resource group
!az group create                                                                             \
    --name $resourceGroupName                                                                \
    --location $resourceLocation                                                             \
    --tag $resourceTags

# Create a public IP address resource with a static IP address
!az network public-ip create                                                                 \
   --name $public_ip_name                                                                    \
   --resource-group $resourceGroupName                                                       \
   --location $resourceLocation                                                              \
   --allocation-method Static                                                                \
   --dns-name $dns_name                                                                      \
   --tag $resourceTags                                                                                                                           

# Create a virtual network with one subnet
!az network vnet create                                                                      \
   --name $vnet_name                                                                         \
   --resource-group $resourceGroupName                                                       \
   --location $resourceLocation                                                              \
   --address-prefix $vnet_prefix                                                             \
   --subnet-name $subnet_name                                                                \
   --subnet-prefix $subnet_prefix                                                            \
   --tag $resourceTags                                                                             


!az network nsg create                                                                       \
   --name $nsg_name                                                                          \
   --resource-group $resourceGroupName                                                       \
   --tag $resourceTags                                                                          

# Open SSH port
!az network nsg rule create                                                                  \
   --resource-group $resourceGroupName                                                       \
   --nsg-name $nsg_name                                                                      \
   --name "Default SSH"                                                                      \
   --destination-port-ranges 22                                                              \
   --protocol Tcp                                                                            \
   --access Allow                                                                            \
   --priority 1020                                                                         

# Create a network interface connected to the VNet with a static private IP address 
# and associate the public IP address resource to the NIC.
!az network nic create                                                                       \
   --name $nic_name                                                                          \
   --resource-group $resourceGroupName                                                       \
   --location $resourceLocation                                                              \
   --subnet $subnet_name                                                                     \
   --vnet-name $vnet_name                                                                    \
   --public-ip-address $public_ip_name                                                       \
   --network-security-group $nsg_name                                                        \
   --tag $resourceTags               


In [ ]:
# Create the VM
# For image param: Ubuntu 18.04-LTS image -> https://docs.microsoft.com/en-us/azure/virtual-machines/linux/cli-ps-findimage
# Instead of Password login, we are uploading Private SSH key. Update the path accrodingly if needed
!az vm create                                                                               \
   --name $iotDeviceId                                                                      \
   --resource-group $resourceGroupName                                                      \
   --location $resourceLocation                                                             \
   --storage-sku Standard_LRS                                                               \
   --os-disk-name $iotDeviceId"_osdisk"                                                     \
   --image "Canonical:UbuntuServer:18.04-LTS:latest"                                        \
   --size $vm_type                                                                          \
   --nics $nic_name                                                                         \
   --admin-username $userName                                                               \
   --authentication-type ssh                                                                \
   --tag $resourceTags                                                                      \
   --generate-ssh-keys                                                                      

## Optional: Remove Previously Trusted Host

If you previously created a VM with the same name, you may run into issues connecting the VM through SSH. As such, you can first remove this VM from your list of known hosts. 

In [ ]:
!ssh-keygen -f ~/.ssh/known_hosts -R $vm_dns_name 

If the above command was unable to run, you can try running the following command in your terminal:

`ssh-keygen -R "<vm_dns_name>"`

where vm_dns_name is the variable defined above. 

## Print the SSH Connection String for the VM

In [ ]:
# If needed, use the output command to SSH into the VM
sshstring = "ssh -i ~/.ssh/id_rsa {0}@{1}".format(userName, vm_dns_name)

In [ ]:
# save the ssh conn string into .env file
set_key(envPath, "SSH_STRING", sshstring.strip())

> <span>[!NOTE]</span>
> If you need the SSH connection string anytime in the future to connect to your IoT Edge device VM, you can find the string in the [.env file](.env) as `SSH_STRING`. 

## Next Steps
If you experience unresolved issues, you can always [delete your resources](https://docs.microsoft.com/en-us/azure/azure-resource-manager/management/delete-resource-group?tabs=azure-powershell) and re-run this section.

If all the code cells above have successfully finished running, return to the Readme page to continue.   